# Using CARTOframes to view earthquakes from the last 30 days

In [1]:
import cartoframes
from cartoframes import Credentials
from cartoframes import Layer, BaseMap, styling
import pandas as pd
%matplotlib inline

In [24]:
USERNAME = 'michellemho-carto'  # <-- replace with your username 
APIKEY = 'abcde'       # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
creds.save()  # save credentials for later use (not dependent on Python session)
cc = cartoframes.CartoContext(creds)

In [2]:
cc = cartoframes.CartoContext()

In [4]:
# Use Pandas to read a CSV from the USGS feed of earthquakes from the last 30 days
earthquakes = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
earthquakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-07T20:20:04.691Z,60.231900,-141.676800,0.10,2.00,ml,NaN,NaN,NaN,0.63,...,2017-11-07T20:26:56.545Z,"45km ENE of Cape Yakataga, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
1,2017-11-07T20:09:49.420Z,33.468500,-116.466667,6.86,1.94,ml,69.0,57.0,0.06458,0.22,...,2017-11-07T20:21:04.820Z,"22km ESE of Anza, CA",earthquake,0.20,0.65,0.173,26.0,automatic,ci,ci
2,2017-11-07T20:08:38.028Z,60.234800,-141.678900,0.00,1.90,ml,NaN,NaN,NaN,0.60,...,2017-11-07T20:12:33.410Z,"45km ENE of Cape Yakataga, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
3,2017-11-07T20:04:26.939Z,58.200800,-151.512900,27.50,2.20,ml,NaN,NaN,NaN,0.52,...,2017-11-07T20:08:42.292Z,"69km NE of Kodiak, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
4,2017-11-07T19:59:39.720Z,33.487333,-116.792833,4.44,0.50,ml,20.0,37.0,0.10520,0.18,...,2017-11-07T20:03:23.319Z,"8km NE of Aguanga, CA",earthquake,0.28,0.86,0.085,15.0,automatic,ci,ci


In [7]:
# Write earthquake data to CARTO
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource
Table successfully written to CARTO: https://michellemho-carto.carto.com/dataset/earthquakes_30


In [8]:
# Map the earthquake data
cc.map(layers=[Layer('earthquakes_30')])

In [24]:
# Style the color and size by the magnitude
cc.map(layers=[Layer('earthquakes_30',
       color={'column': 'mag', 'scheme': styling.redOr(5, bin_method='equal')},
       size={'column':'mag', 'bins':5, 'bin_method':'equal'})])

In [42]:
# Try torque animation of time-stamp column

# Make sure time-stamp column is a pandas date type
earthquakes['time'] = pd.to_datetime(earthquakes.time)

In [54]:
earthquakes.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-07 20:20:04.691,60.2319,-141.676800,0.10,2.00,ml,NaN,NaN,NaN,0.63,...,2017-11-07T20:26:56.545Z,"45km ENE of Cape Yakataga, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
1,2017-11-07 20:09:49.420,33.4685,-116.466667,6.86,1.94,ml,69.0,57.0,0.06458,0.22,...,2017-11-07T20:21:04.820Z,"22km ESE of Anza, CA",earthquake,0.2,0.65,0.173,26.0,automatic,ci,ci


In [55]:
earthquakes.dtypes

time               datetime64[ns]
latitude                  float64
longitude                 float64
depth                     float64
mag                       float64
magType                    object
nst                       float64
gap                       float64
dmin                      float64
rms                       float64
net                        object
id                         object
updated                    object
place                      object
type                       object
horizontalError           float64
depthError                float64
magError                  float64
magNst                    float64
status                     object
locationSource             object
magSource                  object
dtype: object

In [56]:
# Update CARTO dataset
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource
Table successfully written to CARTO: https://michellemho-carto.carto.com/dataset/earthquakes_30


In [61]:
# Create an animated map by specifying the time of the layer
cc.map(layers=[Layer('earthquakes_30', time = {'column':'time', 'method':'sum', 'trails':4, 'duration':30})])

In [60]:
cartoframes.__version__

'0.2.4b4'